# Práctica de Laboratorio: Esteganografía LSB
## Criptografía - Tema 2: Conceptos Básicos

**Estudiante:** TU_NOMBRE_AQUI  
**Fecha:** 8 de Febrero de 2026  

---

Este laboratorio implementa esteganografía LSB (Least Significant Bit) con las siguientes mejoras de seguridad:

- **Cifrado AES-256-GCM**: Todo mensaje se cifra antes de ocultarse
- **Posiciones aleatorias**: No se usa inserción secuencial, sino posiciones determinísticas pseudo-aleatorias
- **KDF (PBKDF2)**: Derivación segura de claves desde contraseña

Esto hace el sistema mucho más robusto que LSB básico.

## PARTE 1: Generador de Dataset

Este código genera 100 imágenes con ruido aleatorio y oculta un mensaje cifrado en UNA de ellas de forma aleatoria.

In [4]:
from PIL import Image
import numpy as np
import os
import random
import hashlib
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes


class SimpleStego:
    """Sistema simplificado de esteganografía LSB con AES"""
    
    def __init__(self, password):
        # Derivar clave AES desde password
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=b'lab_salt_2026',
            iterations=100000
        )
        self.aes_key = kdf.derive(password.encode('utf-8'))
        
        # Derivar seed para posiciones aleatorias
        seed_bytes = hashlib.sha256(self.aes_key).digest()[:4]
        self.seed = int.from_bytes(seed_bytes, 'big')
    
    def encrypt_message(self, message):
        """Cifra mensaje con AES-GCM"""
        aesgcm = AESGCM(self.aes_key)
        nonce = os.urandom(12)
        ciphertext = aesgcm.encrypt(nonce, message.encode('utf-8'), None)
        return nonce + ciphertext
    
    def get_random_positions(self, total_capacity, num_bits):
        """Genera posiciones aleatorias determinísticas"""
        rng = np.random.RandomState(self.seed)
        all_positions = np.arange(total_capacity)
        rng.shuffle(all_positions)
        return all_positions[:num_bits]


def create_noise_image(filename, width=1280, height=720):
    """Crea imagen con píxeles RGB aleatorios"""
    pixels = np.random.randint(0, 256, (height, width, 3), dtype=np.uint8)
    img = Image.fromarray(pixels, 'RGB')
    img.save(filename, 'PNG')


def hide_lsb(image_path, message, password):
    """Oculta mensaje en imagen usando LSB con cifrado y posiciones aleatorias"""
    stego = SimpleStego(password)
    
    # Cifrar mensaje
    encrypted = stego.encrypt_message(message)
    
    # Convertir a binario (con longitud al inicio)
    length_bits = format(len(encrypted), '032b')
    data_bits = ''.join(format(byte, '08b') for byte in encrypted)
    delimiter = '1' * 16
    message_binary = length_bits + data_bits + delimiter
    
    # Abrir imagen
    img = Image.open(image_path)
    pixels = np.array(img)
    height, width = pixels.shape[:2]
    
    # Calcular capacidad
    total_capacity = width * height * 3
    
    if len(message_binary) > total_capacity:
        raise ValueError("Mensaje demasiado grande")
    
    # Generar posiciones aleatorias
    positions = stego.get_random_positions(total_capacity, len(message_binary))
    
    # Insertar bits en LSB
    for i, pos in enumerate(positions):
        pixel_idx = pos // 3
        channel = pos % 3
        y = pixel_idx // width
        x = pixel_idx % width
        
        bit = int(message_binary[i])
        pixels[y, x, channel] = (pixels[y, x, channel] & 0xFE) | bit
    
    # Guardar imagen
    stego_img = Image.fromarray(pixels, 'RGB')
    stego_img.save(image_path, 'PNG')


def generate_dataset(folder, num_images, secret_message, password):
    """Genera dataset completo"""
    os.makedirs(folder, exist_ok=True)
    
    # Generar imágenes con ruido
    print(f"Generando {num_images} imágenes...")
    for i in range(num_images):
        filename = os.path.join(folder, f"image_{i:03d}.png")
        create_noise_image(filename)
    
    # Seleccionar una aleatoriamente
    secret_index = random.randint(0, num_images - 1)
    secret_image = os.path.join(folder, f"image_{secret_index:03d}.png")
    
    # Ocultar mensaje
    print("Ocultando mensaje cifrado...")
    hide_lsb(secret_image, secret_message, password)
    
    print(f"✓ Dataset creado: {num_images} imágenes en '{folder}/'")
    print("✓ Una contiene el mensaje cifrado")


# CONFIGURACIÓN
FOLDER = "dataset_images"
NUM_IMAGES = 100
FLAG = "FLAG{LSB_Stego_2026}"
PASSWORD = "password_lab_2026"

# GENERAR DATASET
generate_dataset(FOLDER, NUM_IMAGES, FLAG, PASSWORD)

Generando 100 imágenes...
Ocultando mensaje cifrado...
✓ Dataset creado: 100 imágenes en 'dataset_images/'
✓ Una contiene el mensaje cifrado


### Explicación del Generador

**¿Cómo funciona?**

1. **Derivación de Claves (KDF)**:
   - Usa PBKDF2 con 100,000 iteraciones para derivar una clave AES-256 desde la contraseña
   - De esta clave deriva un seed determinístico para el generador de posiciones aleatorias

2. **Cifrado del Mensaje**:
   - El mensaje se cifra con AES-GCM antes de ocultarse
   - Esto proporciona confidencialidad e integridad

3. **Posiciones Aleatorias**:
   - No se insertan bits secuencialmente en la imagen
   - Se genera una permutación completa de todas las posiciones disponibles
   - Se usan las primeras N posiciones de esta permutación
   - Esto hace MUCHO más difícil detectar el patrón

4. **Estructura del Payload**:
   ```
   [Longitud: 32 bits] [Datos cifrados: N bytes] [Delimitador: 16 bits]
   ```

**Ventajas sobre LSB básico**:
- Confidencialidad: Mensaje cifrado con AES-256
- Integridad: GCM detecta modificaciones
- Resistencia a análisis: Posiciones aleatorias dificultan detección estadística
- Autenticación: Solo quien conoce la contraseña puede descifrar

## PARTE 2: Detector Forense

Este código implementa dos estrategias de búsqueda del mensaje oculto.

In [5]:
import time

# Reutilizamos la clase SimpleStego definida arriba

def extract_lsb(image_path, num_bits=None, password=None):
    """Extrae bits LSB usando posiciones aleatorias"""
    img = Image.open(image_path)
    pixels = np.array(img)
    height, width = pixels.shape[:2]
    
    total_capacity = width * height * 3
    
    if num_bits is None:
        num_bits = total_capacity
    
    # Obtener posiciones
    stego = SimpleStego(password)
    positions = stego.get_random_positions(total_capacity, num_bits)
    
    # Extraer bits
    extracted_bits = []
    for pos in positions:
        pixel_idx = pos // 3
        channel = pos % 3
        y = pixel_idx // width
        x = pixel_idx % width
        
        bit = pixels[y, x, channel] & 1
        extracted_bits.append(str(bit))
    
    return ''.join(extracted_bits)


def binary_to_bytes(bits):
    """Convierte string de bits a bytes"""
    byte_list = []
    for i in range(0, len(bits), 8):
        byte_bits = bits[i:i+8]
        if len(byte_bits) == 8:
            byte_list.append(int(byte_bits, 2))
    return bytes(byte_list)


def search_brute_force(folder, password, header="FLAG{"):
    """Búsqueda por fuerza bruta - analiza TODAS las imágenes completamente"""
    start_time = time.time()
    
    images = sorted([f for f in os.listdir(folder) if f.endswith('.png')])
    stego = SimpleStego(password)
    
    print(f"Búsqueda fuerza bruta: {len(images)} imágenes")
    
    for i, img_name in enumerate(images):
        img_path = os.path.join(folder, img_name)
        
        if (i + 1) % 20 == 0:
            print(f"  Progreso: {i+1}/{len(images)}")
        
        try:
            # Extraer TODOS los bits
            bits = extract_lsb(img_path, password=password)
            
            # Leer longitud
            if len(bits) < 32:
                continue
            
            data_length = int(bits[:32], 2)
            
            if data_length <= 0 or data_length > 10000:
                continue
            
            # Extraer datos
            data_bits_needed = 32 + (data_length * 8)
            if len(bits) < data_bits_needed:
                continue
            
            data_bits = bits[32:data_bits_needed]
            encrypted_data = binary_to_bytes(data_bits)
            
            # Descifrar
            message = stego.decrypt_message(encrypted_data)
            
            if message and header in message:
                elapsed = time.time() - start_time
                print(f"\n✓ Encontrado en {img_name}")
                return img_name, message, elapsed
                
        except:
            continue
    
    elapsed = time.time() - start_time
    return None, None, elapsed


def search_optimized(folder, password, header="FLAG"):
    """Búsqueda optimizada - solo extrae bytes necesarios"""
    start_time = time.time()
    
    images = sorted([f for f in os.listdir(folder) if f.endswith('.png')])
    stego = SimpleStego(password)
    
    # Solo extraer bits suficientes para verificación inicial
    header_check_bits = 32 + (50 * 8)
    
    print(f"Búsqueda optimizada: {len(images)} imágenes")
    print(f"Extracción inicial: solo {header_check_bits} bits")
    
    for i, img_name in enumerate(images):
        img_path = os.path.join(folder, img_name)
        
        if (i + 1) % 20 == 0:
            print(f"  Progreso: {i+1}/{len(images)}")
        
        try:
            # Paso 1: Extraer solo bits iniciales (RÁPIDO)
            bits_quick = extract_lsb(img_path, num_bits=header_check_bits, password=password)
            
            if len(bits_quick) < 32:
                continue
            
            data_length = int(bits_quick[:32], 2)
            
            if data_length <= 0 or data_length > 10000:
                continue
            
            # Paso 2: Si longitud válida, extraer mensaje completo
            full_bits_needed = 32 + (data_length * 8) + 16
            bits_full = extract_lsb(img_path, num_bits=full_bits_needed, password=password)
            
            data_bits = bits_full[32:32 + (data_length * 8)]
            encrypted_data = binary_to_bytes(data_bits)
            
            # Paso 3: Descifrar y verificar
            message = stego.decrypt_message(encrypted_data)
            
            if message and header in message:
                elapsed = time.time() - start_time
                print(f"\n✓ Encontrado en {img_name}")
                return img_name, message, elapsed
                
        except:
            continue
    
    elapsed = time.time() - start_time
    return None, None, elapsed


# Método para descifrado (necesario para SimpleStego)
def decrypt_message_method(self, encrypted_data):
    try:
        nonce = encrypted_data[:12]
        ciphertext = encrypted_data[12:]
        aesgcm = AESGCM(self.aes_key)
        plaintext = aesgcm.decrypt(nonce, ciphertext, None)
        return plaintext.decode('utf-8')
    except:
        return None

# Añadir método a la clase
SimpleStego.decrypt_message = decrypt_message_method

### Explicación del Detector

**Dos estrategias implementadas:**

**1. Fuerza Bruta (Naive)**:
- Analiza CADA imagen completamente
- Extrae TODOS los bits LSB de cada imagen
- Intenta descifrar y buscar el header "FLAG{LSB_Stego_2026}"
- Complejidad: O(N × W × H) donde N = número de imágenes

**2. Optimizada (Early Termination)**:
- Primero extrae solo los bits necesarios para verificar la longitud
- Si la longitud parece inválida, pasa a la siguiente imagen
- Solo si la longitud es válida extrae el mensaje completo
- Complejidad: O(N × k) donde k << W × H

**¿Por qué la optimizada es más rápida?**

En la mayoría de imágenes (99 de 100), el campo de longitud contendrá basura. Detectamos esto rápido y saltamos a la siguiente imagen sin procesar todos los píxeles.

**Consideraciones con posiciones aleatorias:**

Aunque usamos posiciones aleatorias, seguimos necesitando la contraseña correcta para:
1. Generar las mismas posiciones que usó el encoder
2. Descifrar el mensaje con AES-GCM

Sin la contraseña correcta, es computacionalmente inviable encontrar el mensaje.

## PARTE 3: Experimentos y Medición de Tiempos

Ahora ejecutamos ambos métodos y medimos tiempos.

In [6]:
# EJECUTAR BÚSQUEDAS

PASSWORD = "password_lab_2026"  # Debe coincidir con el generador
FOLDER = "dataset_images"

print("="*60)
print("MÉTODO 1: FUERZA BRUTA")
print("="*60)

img1, msg1, time1 = search_brute_force(FOLDER, PASSWORD)

if msg1:
    print(f"\nMensaje: {msg1}")
    print(f"Tiempo: {time1:.4f} segundos")
else:
    print("\nNo encontrado")
    print(f"Tiempo: {time1:.4f} segundos")

print("\n" + "="*60)
print("MÉTODO 2: OPTIMIZADO")
print("="*60)

img2, msg2, time2 = search_optimized(FOLDER, PASSWORD)

if msg2:
    print(f"\nMensaje: {msg2}")
    print(f"Tiempo: {time2:.4f} segundos")
else:
    print("\nNo encontrado")
    print(f"Tiempo: {time2:.4f} segundos")

print("\n" + "="*60)
print("COMPARACIÓN")
print("="*60)

if time1 > 0 and time2 > 0:
    speedup = time1 / time2
    print(f"Fuerza Bruta: {time1:.4f} seg")
    print(f"Optimizado:   {time2:.4f} seg")
    print(f"Mejora:       {speedup:.2f}x más rápido")

MÉTODO 1: FUERZA BRUTA
Búsqueda fuerza bruta: 100 imágenes
  Progreso: 20/100
  Progreso: 40/100
  Progreso: 60/100

✓ Encontrado en image_065.png

Mensaje: FLAG{LSB_Stego_2026}
Tiempo: 244.7284 segundos

MÉTODO 2: OPTIMIZADO
Búsqueda optimizada: 100 imágenes
Extracción inicial: solo 432 bits
  Progreso: 20/100
  Progreso: 40/100
  Progreso: 60/100

✓ Encontrado en image_065.png

Mensaje: FLAG{LSB_Stego_2026}
Tiempo: 25.0655 segundos

COMPARACIÓN
Fuerza Bruta: 244.7284 seg
Optimizado:   25.0655 seg
Mejora:       9.76x más rápido


## Resultados de Tiempos

| W x H   | Fuerza Bruta (sec) | Optimizado (sec) |
|---------|--------------------|------------------|
| 200x200 | 33.5705            | 1.6054           |
| 500x400 | 8.4581             | 1.3704           |
| 1280x720| 244.7284           | 25.0655          |


## Respuestas a las Preguntas de Análisis

### Pregunta 1: Complejidad Teórica

**a) Complejidad del enfoque de fuerza bruta:**

Sea:
- N = número de imágenes
- W = ancho de imagen
- H = alto de imagen

**Respuesta:** O(N × W × H)

**Justificación:** Para cada una de las N imágenes, debemos:
1. Abrir la imagen: O(W × H)
2. Extraer TODOS los bits LSB: O(W × H × 3)
3. Intentar descifrar: O(m) donde m es el tamaño del mensaje

Como W × H domina sobre m, la complejidad es O(N × W × H).

**b) Complejidad del enfoque optimizado:**

**Respuesta:** O(N × k) en caso promedio, O(N × W × H) en peor caso

Donde k es el número de bits iniciales extraídos (~400 bits para verificación).

**Justificación:** 
- Para 99 de 100 imágenes: solo extraemos k bits → O(k)
- Para 1 imagen (la correcta): extraemos todo → O(W × H)
- Total: O(99k + W×H) ≈ O(N × k) cuando k << W × H

**c) Speedup teórico para W=H=1000, k=5:**

Fuerza bruta: N × 1000 × 1000 = N × 1,000,000 operaciones

Optimizado (caso promedio): N × 40 bytes × 8 bits = N × 320 operaciones

Speedup = 1,000,000 / 320 ≈ **3,125x más rápido**

### Pregunta 2: Cuellos de Botella

**a) ¿Qué tarda más: I/O o CPU?**

**Respuesta:** Depende del tamaño de la imagen.

Para imágenes pequeñas (200×200): El cuello de botella es **I/O** (lectura de archivos).

Para imágenes grandes (1000×1000): El cuello de botella es **CPU** (procesamiento de píxeles y descifrado AES).

**Justificación:**
- Abrir imagen: ~1-5ms (I/O)
- Procesar 40,000 píxeles (200×200): ~5-10ms (CPU)
- Procesar 1,000,000 píxeles (1000×1000): ~100-200ms (CPU)
- Descifrado AES: ~0.1-1ms por intento (CPU)

**b) Verificación empírica:**

```python
import time

# Medir solo I/O
start = time.time()
img = Image.open("test/image.png")
io_time = time.time() - start

# Medir solo CPU
start = time.time()
pixels = np.array(img)
bits = extract_all_lsb(pixels)
cpu_time = time.time() - start

print(f"I/O: {io_time:.4f}s, CPU: {cpu_time:.4f}s")
```

**c) Si el cuello de botella es I/O, ¿ayuda un CPU más rápido?**

**Respuesta:** No significativamente.

Si el 80% del tiempo se gasta en I/O (lectura de disco), mejorar el CPU solo acelera el 20% restante. Por la Ley de Amdahl:

Speedup máximo = 1 / (0.8 + 0.2/S) donde S es la mejora del CPU

Incluso con CPU infinitamente rápido (S→∞): Speedup = 1/0.8 = **1.25x**

**Solución:** Usar SSD en lugar de HDD, o cachear imágenes en RAM.

### Pregunta 3: Escalabilidad

**a) Tiempo estimado para 1 millón de imágenes:**

Asumiendo:
- Tiempo por imagen (optimizado): ~0.01 segundos
- N = 1,000,000 imágenes

Tiempo total = 1,000,000 × 0.01s = **10,000 segundos ≈ 2.78 horas**

Con fuerza bruta sería: ~100 horas (imposible en la práctica).

**b) Uso de multiprocesamiento:**

```python
from multiprocessing import Pool

def analyze_image(img_path):
    # Análisis de una imagen
    return extract_and_check(img_path)

with Pool(8) as pool:  # 8 procesos paralelos
    results = pool.map(analyze_image, image_list)
```

**Ventajas:**
- Cada CPU core procesa imágenes independientemente
- Escalabilidad casi lineal (cada core duplica velocidad)

**c) Complejidad con P procesadores:**

**Respuesta:** O((N × k) / P)

Asumiendo división perfecta del trabajo y sin overhead de sincronización.

En la práctica: O((N × k) / P + overhead)

Speedup real ≈ P × 0.8 (por overhead de comunicación)

Con 8 cores: ~6.4x más rápido
Tiempo para 1M imágenes: 2.78h / 6.4 ≈ **26 minutos**

### Pregunta 4: Seguridad de la Esteganografía

**a) Sin header predecible, ¿es posible automatizar la búsqueda?**

**Respuesta:** Sí, pero es MUCHO más difícil.

Sin cifrado:
- Se puede buscar por entropía (mensajes tienen más estructura que ruido)
- Buscar caracteres ASCII válidos
- Análisis de frecuencia de letras

**CON cifrado AES (nuestro caso):**
- El texto cifrado es indistinguible de ruido aleatorio
- No hay headers, patrones ni estructura detectable
- La única forma es **fuerza bruta de la contraseña**

Complejidad: O(2^k) donde k = bits de entropía de la contraseña

Para contraseña de 12 caracteres aleatorios: 2^72 intentos = **imposible**

**b) Distinguir ruido aleatorio vs mensaje cifrado:**

**Respuesta:** Estadísticamente imposible con cifrado moderno.

AES-GCM produce salida indistinguible de aleatoriedad perfecta. Tests estadísticos:

1. **Chi-cuadrado:** Ambos pasan (distribución uniforme)
2. **Entropía de Shannon:** Ambos ≈ 8 bits/byte (máxima)
3. **Test de compresibilidad:** Ambos incompresibles
4. **Autocorrelación:** Ambos sin patrones

**Única diferencia:** Longitud exacta del payload.

Pero sin conocer la contraseña, no podemos verificar si una longitud dada es correcta o ruido.

**c) Técnicas adicionales del atacante:**

1. **Usar canales de color específicos:** Solo modificar canal azul (menos perceptible)
2. **Adaptive LSB:** Solo modificar píxeles en regiones con alta variación
3. **Spread Spectrum:** Distribuir bits en múltiples imágenes
4. **Steganografía en dominio frecuencia:** Modificar coeficientes DCT en lugar de píxeles
5. **Usar imágenes naturales:** Ruido aleatorio es más sospechoso que fotos reales
6. **Cambiar solo bits ya aleatorios:** Analizar imagen, solo modificar píxeles con LSB "natural"

**Nuestro sistema ya implementa:**
- Posiciones aleatorias (no secuencial)
- Cifrado AES-GCM (indistinguible de ruido)
- KDF robusto (resistente a fuerza bruta)

**Limitación principal:** Usamos imágenes de ruido sintético en lugar de fotos reales, lo cual es sospechoso por sí mismo.

## Conclusiones

Este laboratorio demuestra:

1. **LSB básico es inseguro:** Sin cifrado, cualquiera puede leer el mensaje

2. **Cifrado es esencial:** AES-GCM proporciona confidencialidad e integridad

3. **Posiciones aleatorias mejoran seguridad:** Dificultan análisis estadístico

4. **Optimización reduce complejidad:** De O(N×W×H) a O(N×k)

5. **La seguridad depende de la contraseña:** Con KDF robusto y contraseña fuerte, el sistema es computacionalmente seguro

6. **Trade-off seguridad vs detección:** Cuanto más seguro (cifrado, posiciones aleatorias), más difícil detectar estadísticamente

**Aplicaciones reales:**
- Marcas de agua digitales
- Comunicación encubierta
- Integridad de imágenes forenses

**Limitaciones:**
- Vulnerable a procesamiento de imagen (crop, resize, compresión)
- Requiere canal PNG sin pérdida
- Detectores avanzados (ML) pueden identificar anomalías
